In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
with open('Book.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [3]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
# Generate input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Split into X and y
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [6]:
model = Sequential([
    Embedding(total_words, 100, input_length=max_sequence_len-1),
    Bidirectional(LSTM(150, return_sequences=True)),
    Bidirectional(LSTM(100)),
    Dense(total_words, activation='softmax')
])

C:\Users\Sneha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [7]:
early_stopping = EarlyStopping(
    monitor='loss', 
    patience=5,  
    restore_best_weights=True,  
    verbose=1
)

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
history = model.fit(X, y, epochs=60, verbose=1,callbacks=[early_stopping] )

Epoch 1/60
632/632 ━━━━━━━━━━━━━━━━━━━━ 36s 45ms/step - accuracy: 0.0475 - loss: 6.3608
Epoch 2/60
632/632 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.0619 - loss: 5.7263
Epoch 3/60
632/632 ━━━━━━━━━━━━━━━━━━━━ 38s 40ms/step - accuracy: 0.0735 - loss: 5.5297
Epoch 4/60
632/632 ━━━━━━━━━━━━━━━━━━━━ 26s 41ms/step - accuracy: 0.1018 - loss: 5.3192
Epoch 5/60
632/632 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.1207 - loss: 5.1274
Epoch 6/60
632/632 ━━━━━━━━━━━━━━━━━━━━ 39s 39ms/step - accuracy: 0.1387 - loss: 4.9191
Epoch 7/60
632/632 ━━━━━━━━━━━━━━━━━━━━ 41s 40ms/step - accuracy: 0.1482 - loss: 4.8002
Epoch 8/60
632/632 ━━━━━━━━━━━━━━━━━━━━ 43s 44ms/step - accuracy: 0.1526 - loss: 4.6750
Epoch 9/60
632/632 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.1535 - loss: 4.6071
Epoch 10/60
632/632 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.1650 - loss: 4.4679
Epoch 11/60
632/632 ━━━━━━━━━━━━━━━━━━━━ 30s 47ms/step - accuracy: 0.1677 - loss: 4.3921
Epoch 12/60
632/632 ━━━━━━━━━━

In [10]:
training_accuracy = history.history['accuracy']
print(f"Final Training Accuracy: {training_accuracy[-1] * 100:.2f}%")


Final Training Accuracy: 87.65%


In [12]:
# Prompt the user for seed text
seed_text = input("Enter the initial text (seed text) to start generating: ")
next_words = int(input("Enter the number of words to generate: "))

# Generate text
print("\nGenerating text...\n")
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print("\nGenerated Text:\n")
print(seed_text)


Enter the initial text (seed text) to start generating:  Gregor then turned to look out the window at the
Enter the number of words to generate:  2



Generating text...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 765ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step

Generated Text:

Gregor then turned to look out the window at the dull weather
